# Use Decision Optimization to plan your diet with `ibm-watson-machine-learning`

This notebook facilitates Decision Optimization and Watson Machine Learning services. It contains steps and code to work with [ibm-watson-machine-learning](https://pypi.python.org/pypi/ibm-watson-machine-learning) library available in PyPI repository. It also introduces commands for getting model and training data, persisting model, deploying model and scoring it.

Some familiarity with Python is helpful. This notebook uses Python 3.

## Learning goals

The learning goals of this notebook are:

-  Load a DO model file into an Watson Machine learning repository.
-  Prepare data for training and evaluation.
-  Create an DO machine learning job.
-  Persist a DO model Watson Machine Learning repository.
-  Deploy a model for batch scoring using Wastson Machine Learning API.

## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.  [Download externally created DO model](#download)
3.	[Persist externally created DO model](#persistence)
4.	[Deploy](#deploy)
5.	[Create job](#job)
6.  [Clean up](#cleanup)
7.	[Summary and next steps](#summary)



<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `password`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
password = 'PASTE YOUR PASSWORD HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [2]:
wml_credentials = {
    "username": username,
    "password": password,
    "url": url,
    "instance_id": 'openshift',
    "version": '3.5'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [3]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd3.5/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [5]:
client.set.default_space(space_id)

'SUCCESS'

#### <a id="download"></a>
## 2. Download externally created DO model and data


In this section, you will download externally created DO models and data used for training it.


**Action**: Get your DO model.

In [6]:
!wget https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd3.5/models/decision_optimization/do-model.tar.gz \
    -O  do-model.tar.gz

--2020-12-10 15:53:24--  https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd3.5/models/decision_optimization/do-model.tar.gz
Resolving github.com... 140.82.121.4
Connecting to github.com|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cpd3.5/models/decision_optimization/do-model.tar.gz [following]
--2020-12-10 15:53:25--  https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cpd3.5/models/decision_optimization/do-model.tar.gz
Resolving raw.githubusercontent.com... 151.101.64.133, 151.101.128.133, 151.101.192.133, ...
Connecting to raw.githubusercontent.com|151.101.64.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1299 (1.3K) [application/octet-stream]
Saving to: 'do-model.tar.gz'

do-model.tar.gz     100%[===================>]   1.27K  --.-KB/s    in 0s      

2020-12-10 15:53:25 (61.9 MB/s) - 'd

In [7]:
model_path = 'do-model.tar.gz'


<a id="persistence"></a>
## 3. Persist externally created DO model

In this section, you will learn how to store your model in Watson Machine Learning repository by using the Watson Machine Learning Client.


### 3.1: Publish model


#### Publish model in Watson Machine Learning repository on Cloud.

Define model name, autor name and email.


Get software specification for DO model

In [8]:
sofware_spec_uid = client.software_specifications.get_uid_by_name("do_12.9")

Output data schema for storing model in WML repository

In [9]:
output_data_schema = [{'id': 'stest',
                              'type': 'list',
                              'fields': [{'name': 'age', 'type': 'float'},
                                         {'name': 'sex', 'type': 'float'},
                                         {'name': 'cp', 'type': 'float'},
                                         {'name': 'restbp', 'type': 'float'},
                                         {'name': 'chol', 'type': 'float'},
                                         {'name': 'fbs', 'type': 'float'},
                                         {'name': 'restecg', 'type': 'float'},
                                         {'name': 'thalach', 'type': 'float'},
                                         {'name': 'exang', 'type': 'float'},
                                         {'name': 'oldpeak', 'type': 'float'},
                                         {'name': 'slope', 'type': 'float'},
                                         {'name': 'ca', 'type': 'float'},
                                         {'name': 'thal', 'type': 'float'}]
                               }, {'id': 'teste2',
                 'type': 'test',
                 'fields': [{'name': 'age', 'type': 'float'},
                            {'name': 'sex', 'type': 'float'},
                            {'name': 'cp', 'type': 'float'},
                            {'name': 'restbp', 'type': 'float'},
                            {'name': 'chol', 'type': 'float'},
                            {'name': 'fbs', 'type': 'float'},
                            {'name': 'restecg', 'type': 'float'},
                            {'name': 'thalach', 'type': 'float'},
                            {'name': 'exang', 'type': 'float'},
                            {'name': 'oldpeak', 'type': 'float'},
                            {'name': 'slope', 'type': 'float'},
                            {'name': 'ca', 'type': 'float'},
                            {'name': 'thal', 'type': 'float'}]}]

In [10]:
model_meta_props = {
                        client.repository.ModelMetaNames.NAME: "LOCALLY created DO model",
                        client.repository.ModelMetaNames.TYPE: "do-docplex_12.9",
                        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid,
                        client.repository.ModelMetaNames.OUTPUT_DATA_SCHEMA: output_data_schema
                    }
published_model = client.repository.store_model(model=model_path, meta_props=model_meta_props)

**Note:** You can see that model is successfully stored in Watson Machine Learning Service.

### 3.2: Get model details


In [ ]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

### 3.3 Get all models

In [ ]:
client.repository.list_models()

<a id="deploy"></a>
## 4. Deploy
In this section you will learn how to create batch deployment to create job using the Watson Machine Learning Client.

You can use commands bellow to create batch deployment for stored model (web service).


### 4.1: Create model deployment


In [12]:
meta_data = {
    client.deployments.ConfigurationMetaNames.NAME: "deployment_DO",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name": "S", "num_nodes": 1}

}
deployment_details = client.deployments.create(published_model_uid, meta_props=meta_data)



#######################################################################################

Synchronous deployment creation for uid: 'f3314451-3e7b-4e21-a7cd-2bd491d5c84a' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='a8fd51fb-a7e8-4290-aaea-85221a5bd519'
------------------------------------------------------------------------------------------------




**Note**: Here we use deployment url saved in published_model object. In next section, we show how to retrive deployment url from Watson Machine Learning instance.


In [13]:
deployment_uid = client.deployments.get_uid(deployment_details)

Now, You can list all deployments.

In [ ]:
client.deployments.list()

### 4.2: Get deployment details


In [14]:
client.deployments.get_details(deployment_uid)

{'entity': {'asset': {'id': 'f3314451-3e7b-4e21-a7cd-2bd491d5c84a'},
  'batch': {},
  'custom': {},
  'deployed_asset_type': 'do',
  'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b',
   'name': 'S',
   'num_nodes': 1},
  'name': 'deployment_DO',
  'space_id': '1689725f-8035-42be-a9fe-7d3a61de66ee',
  'status': {'state': 'ready'}},
 'metadata': {'created_at': '2020-12-10T14:54:17.591Z',
  'id': 'a8fd51fb-a7e8-4290-aaea-85221a5bd519',
  'modified_at': '2020-12-10T14:54:17.591Z',
  'name': 'deployment_DO',
  'owner': '1000330999',
  'space_id': '1689725f-8035-42be-a9fe-7d3a61de66ee'}}

<a id="job"></a>
## 5. Create job

You can create job to web-service deployment using `create_job` method.

Prepare test data 

In [15]:
import pandas as pd
diet_food = pd.DataFrame([["Roasted Chicken", 0.84, 0, 10],
                          ["Spaghetti W/ Sauce", 0.78, 0, 10],
                          ["Tomato,Red,Ripe,Raw", 0.27, 0, 10],
                          ["Apple,Raw,W/Skin", 0.24, 0, 10],
                          ["Grapes", 0.32, 0, 10],
                          ["Chocolate Chip Cookies", 0.03, 0, 10],
                          ["Lowfat Milk", 0.23, 0, 10],
                          ["Raisin Brn", 0.34, 0, 10],
                          ["Hotdog", 0.31, 0, 10]], columns=["name", "unit_cost", "qmin", "qmax"])

diet_food_nutrients = pd.DataFrame([
    ["Spaghetti W/ Sauce", 358.2, 80.2, 2.3, 3055.2, 11.6, 58.3, 8.2],
    ["Roasted Chicken", 277.4, 21.9, 1.8, 77.4, 0, 0, 42.2],
    ["Tomato,Red,Ripe,Raw", 25.8, 6.2, 0.6, 766.3, 1.4, 5.7, 1],
    ["Apple,Raw,W/Skin", 81.4, 9.7, 0.2, 73.1, 3.7, 21, 0.3],
    ["Grapes", 15.1, 3.4, 0.1, 24, 0.2, 4.1, 0.2],
    ["Chocolate Chip Cookies", 78.1, 6.2, 0.4, 101.8, 0, 9.3, 0.9],
    ["Lowfat Milk", 121.2, 296.7, 0.1, 500.2, 0, 11.7, 8.1],
    ["Raisin Brn", 115.1, 12.9, 16.8, 1250.2, 4, 27.9, 4],
    ["Hotdog", 242.1, 23.5, 2.3, 0, 0, 18, 10.4]
], columns=["Food", "Calories", "Calcium", "Iron", "Vit_A", "Dietary_Fiber", "Carbohydrates", "Protein"])

diet_nutrients = pd.DataFrame([
    ["Calories", 2000, 2500],
    ["Calcium", 800, 1600],
    ["Iron", 10, 30],
    ["Vit_A", 5000, 50000],
    ["Dietary_Fiber", 25, 100],
    ["Carbohydrates", 0, 300],
    ["Protein", 50, 100]
], columns=["name", "qmin", "qmax"])

In [16]:
job_payload_ref = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [
        {
            "id": "diet_food.csv",
            "values": diet_food
        },
        {
            "id": "diet_food_nutrients.csv",
            "values": diet_food_nutrients
        },
        {
            "id": "diet_nutrients.csv",
            "values": diet_nutrients
        }
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
        {
            "id": ".*.csv"
        }
    ]
}

Create job using Watson Machine Learning client

In [17]:
job = client.deployments.create_job(deployment_uid, meta_props=job_payload_ref)

Checking created job status and calculated KPI.

In [18]:
import time

job_id = client.deployments.get_job_uid(job)

elapsed_time = 0
while client.deployments.get_job_status(job_id).get('state') != 'completed' and elapsed_time < 300:
    elapsed_time += 10
    time.sleep(10)
if client.deployments.get_job_status(job_id).get('state') == 'completed':
    job_details_do = client.deployments.get_job_details(job_id)
    kpi = job_details_do['entity']['decision_optimization']['solve_state']['details']['KPI.Total Calories']
    print(f"KPI: {kpi}")
else:
    print("Job hasn't completed successfully in 5 minutes.")

KPI: 2000.0


<a id="cleanup"></a>
## 6. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd3.5/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 7. Summary and next steps

You successfully completed this notebook! You learned how to use DO as well as Watson Machine Learning for model creation and deployment. 

Check out our _[Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/analyze-data/wml-setup.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 


### Authors

**Wojciech Jargielo**, Software Engineer

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.
